![center](https://wallpapercave.com/wp/wp5836619.jpg)

# Hello and Welcome

I want to introduce you to a simple comparative model analysis for lego classification. I never work with such a small data-set. It's important to understand the Convolution Neural Network nature. I know that every person that has been started to learn the NN concept has been faced with to sentence " Neural Networks need tons of data". Comtratyly its value to understanding why. I think a nice way to understand is comparing models with the same small dataset. In this work, you can see the result of 5 different network architecture on the problem space. Actually there 2 neural networks with either image-net initialized weights or not. Lastly, 1 custom and simple architecture (I called it JARJAR-Net 😜) yet includes some important concepts like Batchnormlization, Dropout, and Maxpooling. I want to discuss why networks need more data and which network has been best fitted for the problem space and of course why? I especially thank you, that who joins this conversation. I hope you will enjoy it.

In [ ]:
from PIL import Image
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0, DenseNet121
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dropout, Add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE = (256,256,3)
PATH = '/kaggle/input/lego-minifigures-classification/'
NB_CLASS = 29

In [ ]:
df = pd.read_csv(PATH + 'index.csv')
df = df.drop(['Unnamed: 0'], axis = 1)

train_df = df[df['train-valid'] == 'train']
validation_df = df[df['train-valid'] == 'valid']

train_df['class_id'] = train_df['class_id'].astype(str)
validation_df['class_id'] = validation_df['class_id'].astype(str)

In [ ]:
base_model = DenseNet121(
                            input_shape=IMG_SHAPE, 
                            include_top=False, 
                            weights='imagenet'
    
                            )

base_model.trainable = True

x = base_model.output

global_average = GlobalAveragePooling2D()(x)
output_layer = Dense(NB_CLASS, activation = 'softmax')(global_average)

denseNet_withW= Model(inputs=base_model.input, outputs=output_layer)

#denseNet_withW.summary()

In [ ]:
base_model = DenseNet121(
                            input_shape=IMG_SHAPE, 
                            include_top=False, 
                            weights= None
    
                            )

base_model.trainable = True

x = base_model.output

global_average = GlobalAveragePooling2D()(x)
output_layer = Dense(NB_CLASS, activation = 'softmax')(global_average)

denseNet_withoW = Model(inputs=base_model.input, outputs=output_layer)

#denseNet_withoW.summary()

## I didn't freeze the layer for transfer learning. I just used imageNet weights for initialization.
### Additionally I selected the B0 model for simplicity.

In [ ]:
base_model = EfficientNetB0(
                            input_shape=IMG_SHAPE, 
                            include_top=False, 
                            weights= 'imagenet'
    
                            )

base_model.trainable = True

x = base_model.output

global_average = GlobalAveragePooling2D()(x)
output_layer = Dense(NB_CLASS, activation = 'softmax')(global_average)

eNet_withW = Model(inputs=base_model.input, outputs=output_layer)

#eNet_withW.summary()

In [ ]:
base_model = EfficientNetB0(
                            input_shape=IMG_SHAPE, 
                            include_top=False, 
                            weights= None
    
                            )

base_model.trainable = True

x = base_model.output

global_average = GlobalAveragePooling2D()(x)
output_layer = Dense(NB_CLASS, activation = 'softmax')(global_average)

eNet_withoW = Model(inputs=base_model.input, outputs=output_layer)

#eNet_withoW.summary()

## This is simple CNN architecture with ;
    * 2-D max pooling for reducing parameter size and help to overcome overfitting problem
    * Batch Normalization to avoid overfitting problem and gradient exploding and vanishing. (It's important for deep neural nets)
    * Relu activation has been selected.

In [ ]:
input_layer = Input(shape = IMG_SHAPE)


x = Conv2D(256, (3,3), activation = 'relu')(input_layer) 
x = MaxPool2D((2,2))(x)


x = Conv2D(256, (3,3), activation = 'relu')(x) 
x = Conv2D(128, (3,3), activation = 'relu')(x)
x = MaxPool2D((3,3))(x)


x = Conv2D(128, (3,3), activation = 'relu')(x)
x = Conv2D(128, (3,3), activation = 'relu')(x)
x = Conv2D(64, (3,3), activation = 'relu')(x) 
x = MaxPool2D((2,2))(x)


x = Conv2D(256, (3,3), activation = 'relu')(x) 
x = Dropout(0.5)(x)
x = Conv2D(256, (3,3), activation = 'relu')(x)
x = Conv2D(32, (3,3), activation = 'relu')(x) 
x = MaxPool2D((2,2))(x)
x = BatchNormalization()(x)

x = Flatten()(x) 
x = Dense(1024, activation = 'relu')(x) 

output_layer = Dense(NB_CLASS, activation = 'softmax')(x)

jarjar_model = Model(inputs = input_layer, outputs = output_layer) 

#jarjar_model.summary()

In [ ]:
train_generator = ImageDataGenerator(
                                     rescale=1./255,
                                     rotation_range=45,
                                     width_shift_range=0.25,
                                     height_shift_range=0.25,
                                     shear_range=0.25,
                                     zoom_range=0.25,
                                     horizontal_flip=True,
                                     brightness_range=[0.5, 1.0], 
                                     

                                    )

validation_generator = ImageDataGenerator(rescale=1./255)



train_set = train_generator.flow_from_dataframe(train_df, 
                                                PATH,
                                                x_col = 'path',
                                                y_col = 'class_id',
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'categorical',
                                                target_size = IMG_SHAPE[:2],
                                                suffle=True
                                               )

validation_set = validation_generator.flow_from_dataframe(validation_df, 
                                                          PATH,
                                                          x_col = 'path',
                                                          y_col = 'class_id',
                                                          batch_size = 1,
                                                          class_mode = 'categorical',
                                                          target_size = IMG_SHAPE[:2],
                                                          shuffle = False
                                               )



In [ ]:
denseNet_withW.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics='accuracy')
history_denseNet_withW = denseNet_withW.fit_generator(train_set, 
                              steps_per_epoch=train_set.n//train_set.batch_size, 
                              validation_data = validation_set, 
                              epochs = 50)


In [ ]:
denseNet_withoW.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics='accuracy')
history_denseNet_withoW = denseNet_withoW.fit_generator(train_set, 
                              steps_per_epoch=train_set.n//train_set.batch_size, 
                              validation_data = validation_set, 
                              epochs = 100)

In [ ]:
eNet_withoW.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics='accuracy')
history_eNet_withoW = eNet_withoW.fit_generator(train_set, 
                              steps_per_epoch=train_set.n//train_set.batch_size, 
                              validation_data = validation_set, 
                              epochs = 100)

In [ ]:
eNet_withW.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics='accuracy')
history_eNet_withW = eNet_withW.fit_generator(train_set, 
                              steps_per_epoch=train_set.n//train_set.batch_size, 
                              validation_data = validation_set, 
                              epochs = 50)

In [ ]:
jarjar_model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics='accuracy')
history_jarjar = jarjar_model.fit_generator(train_set, 
                              steps_per_epoch=train_set.n//train_set.batch_size, 
                              validation_data = validation_set, 
                              epochs = 300)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_jarjar.history['accuracy'])
plt.plot(history_jarjar.history['val_accuracy'])
plt.title('JARJAR-NET ACCURACY PLOT')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

plt.plot(history_denseNet_withW.history['accuracy'])
plt.plot(history_denseNet_withW.history['val_accuracy'])
plt.title('DENSE-NET WITH IMAGENET ACCURACY PLOT')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

plt.plot(history_denseNet_withoW.history['accuracy'])
plt.plot(history_denseNet_withoW.history['val_accuracy'])
plt.title('DENSE-NET WITHOUT ACCURACY PLOT')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

plt.plot(history_eNet_withW.history['accuracy'])
plt.plot(history_eNet_withW.history['val_accuracy'])
plt.title('EFFICIENT-NET WITH IMAGENET ACCURACY')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

plt.plot(history_eNet_withoW.history['accuracy'])
plt.plot(history_eNet_withoW.history['val_accuracy'])
plt.title('EFFICIENT-NET WITHOUT IMAGENET ACCURACY PLOT')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Conclusion

It is interesting to see the result of the models. Maybe it will be emotional but as though every model has its own language and characteristics. When we look at the E-Net model, the model quickly fits the training data. It's good if your training and validation set is big and both of them come from similar distribution. Also, we can say that this is complex to understand tiny patterns in the data. We can say the same thing for denseNet. But for Jarjar Net its clear to see JarJar himself as a model. It learns slowly and has oscillation in the learning phase. But in the end, it gives the best result for the validation set. Also, graphs are proving that

## Future Work

It's interesting to see Siamese Network for these problems. A similarity-based network can perform better than direct classifier models.

![](https://64.media.tumblr.com/4a9f1e246f624e0d09c572516f6dab46/22b6215d00ec5863-df/s1280x1920/c4cbc1cee5d4b25c843c0e825e1233961fdebbac.gifv)